In [48]:
# Libraries
import tkinter as tk
from tkinter import messagebox
import serial  
import json
import os
import time

In [49]:
# Parametri e variabili globali
usbport = 'COM5'
arduino = None
movements = []

## Functions

In [50]:
# This 2 functions maps an input value (in range 0-100) to the corresponding value on the servo, accordingly to the 
# configuration on the json file
def mapping(thumb_big_value, thumb_little_value, index_finger_value, middle_finger_value, ringPinky_value, forearm_value):
    with open("config.json", "r") as json_file:
    #Load the contents of the JSON file into a Python dictionary
        data = json.load(json_file)
    
    #thumb - big servo
    thumb_big = data["thumb_big"]
    
    #thumb - little servo
    thumb_little = data["thumb_little"]
    
    #index_finger servo
    index_finger = data["index_finger"]

    #middle_finger servo
    middle_finger = data["middle_finger"]
 
    #ring_finger servo
    ring_pinky = data["ring_pinky"]
  
    #forearm servo
    forearm = data["forearm"]

    #All inputs
    input_values = [thumb_big_value, thumb_little_value, index_finger_value, middle_finger_value, ringPinky_value, forearm_value]
    #All range
    fingers_data = [thumb_big, thumb_little, index_finger, middle_finger, ring_pinky,forearm]
    fingers_data_mapped = []
    i = 0
    for single_finger in fingers_data:
        start = single_finger["range_from"]
        stop = single_finger["range_to"]
        #print(single_finger)
        fingers_data_mapped.append(calculus(input_values[i],start,stop))
        i = i+1
        
    return fingers_data_mapped
        
def calculus(val,start,stop):
    if start==0:
        return int((val/100)*stop)
    else:
        new_stop = stop - start
        return int(((val/100)*new_stop)+start)  

In [51]:
#******************Funzione per salvare un pacchetto o un movimento ***************************
#Struttura di salvataggio: thumb_big, thumb_little, index, middle, ringPinky, forearm
# movement deve essere una list di list (matrice)
def save_movement(name, movement):
    
    # Carica i dati esistenti dal file JSON se esiste
    # Se il file json non esiste, lo crea e ci aggiunge il nuovo movimento
    dati = {}
    if os.path.exists("movements.json"):
        #Se il file esiste, apro il file
        with open("movements.json", 'r') as file_json:
            dati = json.load(file_json)
            
        # Verifica se il nome è già presente nei dati
        if name in dati:
            print(f"Un movimento con il nome '{name}' esiste già. L'aggiunta verrà annullata")
            return False
        else:
            # Aggiungi la nuova matrice ai dati
            dati[name] = movement

            # Salva il dizionario aggiornato in formato JSON nel file specificato
            with open("movements.json", "w") as json_file:
                json.dump(dati, json_file)
                print(f"Movimento '{name}' aggiunto con successo al file")
                return True
    else:
        dati[name] = movement
        with open("movements.json", "w") as json_file:
            json.dump(dati, json_file)
            print(f"Movimento '{name}' aggiunto con successo al file")
            return True

In [52]:
# Apertura connessione seriale
def open_serial_port():
    global arduino 
    try:
        if arduino is None or not arduino.is_open:
            arduino = serial.Serial(port='COM5', baudrate=115200, timeout=.4) 
            time.sleep(2)
            return True
    except serial.SerialException as e:
        print("Error while opening the serial port:", e)
        messagebox.showerror("Error", "Error while opening the serial port")
        return False
        

In [53]:
# Funzione per la validazione dei campi di input della GUI
def on_validate(action, index, value_if_allowed, prior_value, text, validation_type, trigger_type, widget_name):
    if action == '1':  # Inserimento di un carattere
        if text in '0123456789.-+':
            try:
                # Converti il valore in un int
                float_value = int(value_if_allowed)
                # Verifica se il valore è compreso tra 0 e 100
                if 0 <= float_value <= 100:
                    return True
                else:
                    return False
            except ValueError:
                return False
        else:
            return False
    else:
        return True


In [54]:
# FRAME 1
# Funzione per il pulsante esegui movimento nel FRAME1 della GUI
def on_submit(gui_instance):
    packet = []
    for entry in gui_instance.entry_list1:
        try:
            value = int(entry.get())
            #print("Stampa valore in on_submit frame1")
            #print(value)
            packet.append(value)
        except Exception as e:
            messagebox.showerror("Error", "Enter valid numeric values in all boxes")
            print("ERRORE",e)
            return
    
    result = open_serial_port() #apertura seriale
    print(result)
    #if result is True:
        #messagebox.showinfo("Successo", f"Hai inserito i valori numerici: {packet}")
    
    # Comunicazione ad arduino dei valori
    # Mapping: thumb big - thumb little - index - middle - ring&pinky - forearm
    fingers_data_mapped = mapping(packet[0],packet[1],packet[2],packet[3],packet[4],packet[5])
    # Send values to arduino
    global arduino
    #print("Valori di python\n")
    #print(fingers_data_mapped)
    for value in fingers_data_mapped:
        arduino.write(bytes(str(value), 'utf-8')) 
        time.sleep(0.1) 
        #print(arduino.readline())
    

In [55]:
# FRAME 2
# Funzione per il salvataggio dei movimenti in frame2
def on_save(gui_instance,canvas):
    
    #Se l'utente
    if len(movements) == 0:
        messagebox.showerror("Error", "Enter at least one value package")
        return
        
    # Funzione on_ok viene chiamata quando si preme ok nella finestra di input per inserire il nome movimento
    def on_ok():
        input_text = entry.get()
        if input_text:
            #salvataggio nel json
            if(save_movement(input_text,movements) is False):
                messagebox.showinfo("Info", f"Un movimento con questo nome esiste già")
                input_window.destroy()
                #movements.clear()
                #canvas.delete("all") #pulizia del canvas 
            else: 
                messagebox.showinfo("Info", f"Hai inserito il movimento: {input_text}")
                input_window.destroy()
                movements.clear()
                canvas.delete("all") #pulizia del canvas 
            
    # Creazione della finestra di input
    input_window = tk.Toplevel(gui_instance)
    input_window.title("Enter movement name")
    input_window.geometry("400x150")
    
    # Widget Entry per inserire il testo
    entry = tk.Entry(input_window)
    entry.pack(pady=10)

    # Pulsante "OK" per confermare l'input
    ok_button = tk.Button(input_window, text="OK", command=on_ok)
    ok_button.pack(pady=10)
    

# Funzione per i valori successivi in frame2    
def on_next(gui_instance,canvas):
    packet = []
    for entry in gui_instance.entry_list2:
        try:
            value = int(entry.get())
            #print("valore in on_next frame2")
            #print(value)
            packet.append(value)
        except Exception as e:
            messagebox.showerror("Error", "Enter valid numeric values in all boxes")
            print("ERRORE",e)
            return
    movements.append(packet)
    fill_canvas(movements,canvas,40,21) #parametri
    #aggiornamento del canvas e della scrollbar
    canvas.update_idletasks()
    canvas.config(scrollregion=canvas.bbox("all"))
    
    
# Riempie il canvas con i valori
def fill_canvas(movements,canvas,larghezza_cella, altezza_cella):
    canvas.delete("all") #pulizia del canvas 
    num_colonne = 150 # num_colonne sono gli istanti di tempo
    num_righe = 6 # le 6 righe sono i valori di ogni servomotore 
    r=0
    c=0
    #disposizione nel canvas
    for colonna in movements:
        for riga in colonna:
            x0 = c * larghezza_cella
            y0 = r * altezza_cella
            x1 = x0 + larghezza_cella
            y1 = y0 + altezza_cella
            canvas.create_text((x0 + x1) // 2, (y0 + y1) // 2, text=str(riga))
            r = r+1
        c = c+1
        r = 0

In [61]:
# FRAME 3

#legge valori dal movements.json che contiene i movimenti salvati
def read_json():
    try:
        with open("movements.json", 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print("File not found")
        return None
    except json.JSONDecodeError as e:
        print("Parsing error:", e)
        return None

# Esecuzione di un movimento salvato - 
def execute_movement(movement):
    print("Esecuzione")
    open_serial_port() #apertura seriale
    
    for packet in movement:
        # Comunicazione ad arduino dei valori
        # Mapping: thumb big - thumb little - index - middle - ring&pinky - forearm
        fingers_data_mapped = mapping(packet[0],packet[1],packet[2],packet[3],packet[4],packet[5])
        for value in fingers_data_mapped:
            # Send values to arduino
            global arduino
            #print("Valori di python\n")
            #print(fingers_data_mapped)
            for value in fingers_data_mapped:
                arduino.write(bytes(str(value), 'utf-8')) 
                time.sleep(0.1) 
                print(arduino.readline())
    

## GUI

In [ ]:
class GUI(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Hand movement GUI")
        self.geometry("700x700")

        self.columnconfigure(0, weight=1)  # Prima colonna
        self.columnconfigure(1, weight=1)  # Seconda colonna
        
        # Creazione dei frame
        self.frame1 = tk.Frame(self)
        self.frame2 = tk.Frame(self)
        self.frame3 = tk.Frame(self)
        
        # Configurazione dei frame
        self.configure_frame1()
        self.configure_frame2()
        self.configure_frame3()
        
        # Configurazione menù
        self.configure_menu()
        
        
        # Mostra il primo frame all'avvio
        self.show_frame(self.frame1)

    # *************************************** FRAME 1 CONFIGURATION ***************************************************
    def configure_frame1(self):
        
        title = tk.Label(self.frame1, text="Rapid movement",font="8")
        title.grid(row=0,column=1,pady=40,sticky="w")
        validate_cmd = self.frame1.register(on_validate)
        
        
        # Text1
        label1 = tk.Label(self.frame1, text="Thumb - big servo")
        label1.grid(row=1,column=0)
        thumb_big = tk.Entry(self.frame1, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_big.insert(0, "Value") 
        thumb_big.configure(justify=tk.CENTER) 
        thumb_big.grid(row=1,column=1)
        
        # Text2
        label2 = tk.Label(self.frame1, text="Thumb - little servo")
        label2.grid(row=2,column=0,padx=10)
        thumb_little = tk.Entry(self.frame1, fg='black',validate="key", 
                                validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_little.insert(0, "Value") 
        thumb_little.configure(justify=tk.CENTER) 
        thumb_little.grid(row=2,column=1,pady=5)

        
        # Text3
        label3 = tk.Label(self.frame1, text="Index finger")
        label3.grid(row=3,column=0,padx=10)
        index_finger = tk.Entry(self.frame1, fg='black',validate="key", validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        index_finger.insert(0, "Value") 
        index_finger.configure(justify=tk.CENTER) 
        index_finger.grid(row=3,column=1,pady=5)
        
        # Text4
        label4 = tk.Label(self.frame1, text="Middle finger")
        label4.grid(row=4,column=0)
        middle_finger = tk.Entry(self.frame1, fg='black',validate="key", 
                                 validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        middle_finger.insert(0, "Value") 
        middle_finger.configure(justify=tk.CENTER) 
        middle_finger.grid(row=4,column=1,pady=5)
        # Text5
        label5 = tk.Label(self.frame1, text="Ring and Pinky")
        label5.grid(row=5,column=0)
        ring_pinky = tk.Entry(self.frame1, fg='black',validate="key", 
                              validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        ring_pinky.insert(0, "Value") 
        ring_pinky.configure(justify=tk.CENTER) 
        ring_pinky.grid(row=5,column=1,pady=5)
        
        # Text6
        label6 = tk.Label(self.frame1, text="Forearm")
        label6.grid(row=6,column=0)
        forearm = tk.Entry(self.frame1, fg='black',validate="key", 
                           validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        forearm.insert(0, "Value") 
        forearm.configure(justify=tk.CENTER) 
        forearm.grid(row=6,column=1,pady=5)
        
        #Valori dei campi innestati nell'istanza self
        self.entry_list1 = [thumb_big, thumb_little, index_finger, middle_finger, ring_pinky,forearm]
        
        # Button
        button1 = tk.Button(self.frame1, text="Execute", command=lambda: on_submit(self))
        button1.grid(row=7,column=1)
        
        

    # **************************************** FRAME 2 CONFIGURATION *************************************************
    def configure_frame2(self):
        
        title = tk.Label(self.frame2, text="Create movement",font="8")
        title.grid(row=0,column=1,pady=20,sticky="ew")
        validate_cmd = self.frame2.register(on_validate)
        
        
        # Thumb - big servo
        label1 = tk.Label(self.frame2, text="Thumb - big servo")
        label1.grid(row=1,column=0,sticky='e')
        thumb_big = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_big.insert(0, "Value") 
        thumb_big.configure(justify=tk.CENTER) 
        thumb_big.grid(row=1,column=1)
        
        # Thumb - little servo
        label2 = tk.Label(self.frame2, text="Thumb - little servo")
        label2.grid(row=2,column=0,padx=10,sticky='e')
        thumb_little = tk.Entry(self.frame2, fg='black',validate="key", 
                                validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_little.insert(0, "Value") 
        thumb_little.configure(justify=tk.CENTER) 
        thumb_little.grid(row=2,column=1,pady=5)

        
        # Index finger
        label3 = tk.Label(self.frame2, text="Index finger")
        label3.grid(row=3,column=0,padx=10,sticky='e')
        index_finger = tk.Entry(self.frame2, fg='black',validate="key", 
                                validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        index_finger.insert(0, "Value") 
        index_finger.configure(justify=tk.CENTER) 
        index_finger.grid(row=3,column=1,pady=5)
        
        # Middle finger
        label4 = tk.Label(self.frame2, text="Middle finger")
        label4.grid(row=4,column=0,sticky='e')
        middle_finger = tk.Entry(self.frame2, fg='black',validate="key", 
                                 validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        middle_finger.insert(0, "Value") 
        middle_finger.configure(justify=tk.CENTER) 
        middle_finger.grid(row=4,column=1,pady=5)
        
        # Ring and pinky
        label5 = tk.Label(self.frame2, text="Ring and Pinky")
        label5.grid(row=5,column=0,sticky='e')
        ring_pinky = tk.Entry(self.frame2, fg='black',validate="key", 
                              validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        ring_pinky.insert(0, "Value") 
        ring_pinky.configure(justify=tk.CENTER) 
        ring_pinky.grid(row=5,column=1,pady=5)
        
        # Forearm
        label6 = tk.Label(self.frame2, text="Forearm")
        label6.grid(row=6,column=0,sticky='e')
        forearm = tk.Entry(self.frame2, fg='black',validate="key", 
                           validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        forearm.insert(0, "Value") 
        forearm.configure(justify=tk.CENTER) 
        forearm.grid(row=6,column=1,pady=5)
        
        #Valori dei campi innestati nell'istanza self
        self.entry_list2 = [thumb_big, thumb_little, index_finger, middle_finger, ring_pinky, forearm]
        
        # Frame interno al frame2
        self.inner_frame = tk.Frame(self.frame2)
        self.inner_frame.grid(row=8,column=0)
        l1 = tk.Label(self.inner_frame, text="VALUES")
        l1.grid(row=0,sticky='w')
        l1 = tk.Label(self.inner_frame, text="Thumb - big servo:")
        l1.grid(row=1,sticky='w')
        l2 = tk.Label(self.inner_frame, text="Thumb - little servo:")
        l2.grid(row=2,sticky='w')
        l3 = tk.Label(self.inner_frame, text="Index:")
        l3.grid(row=3,sticky='w')
        l4 = tk.Label(self.inner_frame, text="Middle:")
        l4.grid(row=4,sticky='w')
        l5 = tk.Label(self.inner_frame, text="Ring and pinky:")
        l5.grid(row=5,sticky='w')
        l2 = tk.Label(self.inner_frame, text="Forearm:")
        l2.grid(row=6,sticky='w')
        
        # Creazione di un canvas per i valori inseriti man mano
        canvas = tk.Canvas(self.frame2,width=500, height=200)
        canvas.grid(row=8, column=1, sticky="nsew",pady=(101,0))

        # Creazione di una scrollbar orizzontale
        scrollbar_x = tk.Scrollbar(self.frame2, orient="horizontal", command=canvas.xview)
        scrollbar_x.grid(row=9, column=1, sticky="ew")
        # Configurazione del canvas per l'associazione alla scrollbar orizzontale
        canvas.config(xscrollcommand=scrollbar_x.set)

        # Configura la scrollbar quando il canvas viene ridimensionato
        canvas.bind("<Configure>", self.on_canvas_configure)
        
        
        # Buttons
        button1 = tk.Button(self.frame2, text="Save", command=lambda: on_save(self,canvas))
        button1.grid(row=7,column=1,sticky='e',pady=15)
        
        button2 = tk.Button(self.frame2, text="Add", command=lambda: on_next(self,canvas))
        button2.grid(row=7,column=1,sticky='w',pady=15)

    # *************************************** FRAME 3 CONFIGURATION **********************************************
    def configure_frame3(self):
        #print("Sono in frame 3")
        title = tk.Label(self.frame3, text="Saved movements",font="8")
        title.grid(row=0,column=1,pady=20,sticky="ew")
        
        movements_dict = read_json()
        if(movements_dict is None): 
            return
        index = 1
        for key,value in movements_dict.items():
            label = tk.Label(self.frame3, text=str(key))
            label.grid(row=index, column=0, padx=10, pady=5)

            button = tk.Button(self.frame3, text="Execute", command=lambda: execute_movement(value))
            button.grid(row=index, column=1, padx=10, pady=5)
            index = index + 1 
        

    
    # Funzione per l'evento della scrollbar del canvas
    def on_canvas_configure(self, event):
        canvas = event.widget
        canvas.configure(scrollregion=canvas.bbox("all"))
        
    # Funzione per configurare il menù della GUI 
    def configure_menu(self):
        # Creazione del menu
        menubar = tk.Menu(self)
        self.config(menu=menubar)
        frame_menu = tk.Menu(menubar, tearoff=0)
        frame_menu.add_command(label="Rapid movement", command=lambda: self.show_frame(self.frame1))
        frame_menu.add_command(label="Create movement", command=lambda: self.show_frame(self.frame2))
        frame_menu.add_command(label="Saved movements", command=lambda: self.show_frame(self.frame3))
        frame_menu.add_command(label="Shutdown", command=lambda: self.close())

        menubar.add_cascade(label="Menù", menu=frame_menu)

    # Nasconde tutti i frame e mostra solo quello specificato
    def show_frame(self, frame):
        #Riaggiorno il frame3
        self.configure_frame3()
        
        self.frame1.grid_forget()
        self.frame2.grid_forget()
        self.frame3.grid_forget()
        frame.grid()
       
    #Chiusura programma e com. seriale
    def close(self):
        #print("Sto chiudendo tutto")
        global arduino
        if arduino is not None:
            arduino.close()
            print("Closed serial connection")
        self.destroy()         
        
        
if __name__ == "__main__":
    app = GUI()
    app.mainloop()


Esecuzione
b'1'
b'2'
b'3'
b'4'
b'5'
b'6'
b'1'
b'2'
b'3'
b'4'
b'5'
b'6'
b'1'
b'2'
b'3'
b'4'


In [45]:
movements.clear()

In [47]:
arduino.close()